<a href="https://colab.research.google.com/github/Piyush47200/Supply-Chain/blob/main/Supplier_chain_network_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Limitation: Only 1 port.

Install libraries

In [1]:
!pip install geopy
!pip install openrouteservice
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 85.0 MB/s eta 0:00:00


Import libraries

In [2]:
from geopy.geocoders import Nominatim
import pandas as pd
import time
import openrouteservice
import math

Load port, DC and customer sheets.

In [113]:
df_port_city=pd.read_excel('Port.xlsx')
df_DC_city=pd.read_excel('DC.xlsx')
df_customer_city=pd.read_excel('Customer.xlsx')

Defining dataframes for port, DC and customer to hold coordinates data.

In [114]:
df_coordinate_port_city=pd.DataFrame(columns=['City','Latitude','Longitude'])
df_coordinate_DC_city=pd.DataFrame(columns=['City','Latitude','Longitude'])
df_coordinate_customer_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude', 'Demand'])

Finding coordinates for ports

In [115]:
geolocator = Nominatim(user_agent="your_email_or_app_name")
for i in range(0, len(df_port_city)):
  location = geolocator.geocode(df_port_city['City'].iloc[i])
  if location:
    df_coordinate_port_city.loc[i] = [df_port_city['City'].iloc[i], location.latitude, location.longitude]
  else:
    df_coordinate_port_city.loc[i] = [df_port_city['City'].iloc[i], None, None]

Finding coordinates for DC

In [116]:
import pandas as pd
from geopy.geocoders import Nominatim

if df_coordinate_DC_city.empty:
  df_coordinate_DC_city = pd.DataFrame(columns=['City', 'Latitude', 'Longitude', 'Fixed Cost'])

for i in range(0, len(df_DC_city)):
  location = geolocator.geocode(df_DC_city['City'].iloc[i])
  if location:
    df_coordinate_DC_city.loc[i] = [df_DC_city['City'].iloc[i], location.latitude, location.longitude, df_DC_city['Fixed Cost'].iloc[i]]
  else:
    df_coordinate_DC_city.loc[i] = [df_DC_city['City'].iloc[i], None, None, df_DC_city['Fixed Cost'].iloc[i]]

Finding coordinates for customer

In [117]:
geolocator = Nominatim(user_agent="your_email_or_app_name")

for i in range(0, len(df_customer_city)):
  try:
    location = geolocator.geocode(df_customer_city['City'].iloc[i], timeout=10)
    if location:
      df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], location.latitude, location.longitude,df_customer_city['Demand'].iloc[i]]
    else:
      df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
    #time.sleep(2) # Wait 2 seconds between requests to avoid rate limiting
  except GeocoderTimedOut:
    print("Geocoding timed out for city:", df_customer_city['City'].iloc[i])
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
  except GeocoderUnavailable:
    print("Geocoder service unavailable for city:", df_customer_city['City'].iloc[i])
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]
  except Exception as e:
    print("Error geocoding city:", df_customer_city['City'].iloc[i], "-", e)
    df_coordinate_customer_city.loc[i] = [df_customer_city['City'].iloc[i], None, None,df_customer_city['Demand'].iloc[i]]

Exporting coordintates of customers to excel.

In [118]:
df_coordinate_customer_city.to_excel('coordinate_customer_city.xlsx', index=False)

In [119]:
print(df_coordinate_customer_city)

          City   Latitude  Longitude  Demand
0      Hanover  52.374478   9.738553    1000
1      Rostock  54.088671  12.140021    1000
2         Kiel  54.322708  10.135555    1000
3        Trier  49.759621   6.644188    1000
4    Karlsruhe  49.006870   8.403420    1000
5    Rosenheim  47.853927  12.127262    1000
6   Regensburg  49.019533  12.097487    1000
7    Nuremberg  49.453872  11.077298    1000
8      Dresden  51.049329  13.738144    1000
9      Leipzig  51.340632  12.374733    1000
10   Magdeburg  52.131589  11.639961    1000
11   Frankfurt  50.110644   8.682092    1000
12   Darmstadt  49.872775   8.651177    1000
13    Mannheim  49.489291   8.467310    1000
14     Potsdam  52.400931  13.059140    1000
15       Essen  51.458224   7.015817    1000
16    Dortmund  51.514227   7.465279    1000
17      Kassel  51.315455   9.492410    1000
18         Ulm  48.398497   9.991246    1000
19     Kempten  47.726706  10.316883    1000
20     Marburg  50.809011   8.770470    1000
21        

In [120]:
print(df_coordinate_port_city)

           City   Latitude  Longitude
0  Hamburg port  53.546728   9.969076


In [121]:
print(df_coordinate_DC_city)

        City   Latitude  Longitude  Fixed Cost
0     Bremen  53.075820   8.807165    10000000
1     Berlin  52.510885  13.398937    10000000
2  Stuttgart  48.778449   9.180013    10000000


Defining two new dataframes for distance calculations

In [122]:
df_distance_port_DC=pd.DataFrame(columns=['Source','Destination','Distance'])
df_distance_DC_customer=pd.DataFrame(columns=['Source','Destination','Distance'])

Defining a function which will calculate the distance

In [123]:

def distance(lat1, lon1, lat2, lon2):
  client = openrouteservice.Client(key='5b3ce3597851110001cf62489d683e3d831942f3adab13e6a2e5a7dc')
  source = (lon1, lat1)
  destination = (lon2, lat2)

  try:
    routes = client.directions(coordinates=[source, destination], profile='driving-hgv', format='geojson')
    distance_meters = routes['features'][0]['properties']['segments'][0]['distance']
    distance_km = distance_meters / 1000
    return distance_km
  except openrouteservice.exceptions.ApiError as e:
    print(f"Error finding route between {source} and {destination}: {e}")
    return -1

Estimating distance for port to DC

In [124]:
k=0
for j in range(0,len(df_coordinate_port_city)):
  for i in range(0,len(df_coordinate_DC_city)):
    dist=distance(df_coordinate_port_city['Latitude'].iloc[j], df_coordinate_port_city['Longitude'].iloc[j], df_coordinate_DC_city['Latitude'].iloc[i], df_coordinate_DC_city['Longitude'].iloc[i])
    df_distance_port_DC.loc[k]=[df_coordinate_port_city['City'].iloc[j],df_coordinate_DC_city['City'].iloc[i],dist]
    k=k+1


In [125]:
print(df_distance_port_DC)

         Source Destination  Distance
0  Hamburg port      Bremen  120.5309
1  Hamburg port      Berlin  291.6815
2  Hamburg port   Stuttgart  656.5074


Estimating distance for DC to customer

In [126]:
k=0
for j in range(0,len(df_coordinate_DC_city)):
  for i in range(0,len(df_coordinate_customer_city)):
    dist=distance(df_coordinate_DC_city['Latitude'].iloc[j], df_coordinate_DC_city['Longitude'].iloc[j], df_coordinate_customer_city['Latitude'].iloc[i], df_coordinate_customer_city['Longitude'].iloc[i])
    df_distance_DC_customer.loc[k]=[df_coordinate_DC_city['City'].iloc[j],df_coordinate_customer_city['City'].iloc[i],dist]
    k=k+1

/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.10/dist-packages/openrouteservice/client.py:211: UserWarning: Rat

In [127]:
print(df_distance_DC_customer)

       Source Destination  Distance
0      Bremen     Hanover  134.7738
1      Bremen     Rostock  299.8055
2      Bremen        Kiel  242.4310
3      Bremen       Trier  505.5842
4      Bremen   Karlsruhe  603.0970
..        ...         ...       ...
79  Stuttgart   Oldenburg  627.7750
80  Stuttgart   Flensburg  834.7587
81  Stuttgart    Schwerin  758.2404
82  Stuttgart     Cottbus  633.8279
83  Stuttgart    Chemnitz  433.2494

[84 rows x 3 columns]


In [128]:
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("MyModel")

In [129]:
params = {
"WLSACCESSID": '803f8182-ebee-4591-8370-b64a82f775ea',
"WLSSECRET": '7696ca12-c44b-46e1-9875-a948ff7c3fdf',
"LICENSEID": 2574149,
}
env = gp.Env(params=params)
model = gp.Model(env=env)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2574149
Academic license 2574149 - for non-commercial use only - registered to pi___@gmail.com


Defining known variables

In [281]:
c = 100
r = 0
for i in range(0, len(df_customer_city)):
  r = r + df_customer_city['Demand'][i]

Defining unknown variables

In [282]:
x_var = {}
y_var = {}
z_var = {}

In [283]:
z = 0
for i in range(0, len(df_DC_city)):
  z_var[f"z[{i}]"] = model.addVar(vtype=GRB.BINARY, name=f"z[{i}]")
  z = z + z_var[f"z[{i}]"]

In [284]:
x = 0
for j in range(0, len(df_DC_city)):
  for i in range(0, len(df_port_city)):
    x_var[f"x[{i+j}]"] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"x[{i+j}]")
    x = x + x_var[f"x[{i+j}]"]*z_var[f"z[{j}]"]

In [285]:
y = 0
for j in range(0, len(df_DC_city)):
  for i in range(0, len(df_DC_city)*len(df_customer_city)):
    y_var[f"y[{i}]"] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"y[{i}]")
    y = y + y_var[f"y[{i}]"]*z_var[f"z[{j}]"]

Constraints

Demand of each customer is exactly met from DCs.

In [286]:
for j in range(0, len(df_customer_city)):
  a = 0
  for i in range(0, len(df_DC_city)):
    a = a + y_var[f"y[{j+len(df_customer_city)*i}]"]
  model.addConstr(a == df_customer_city['Demand'][j])

1. Outgoing from ports = total demand
2. Outgoing from DC = total demand
3. atleast one DC is opened

In [287]:
model.addConstr(x == r, "Demand_x")
model.addConstr(y == r, "Demand_y")
model.addConstr(z >= 1, "Z_non_zero")

<gurobi.Constr *Awaiting Model Update*>

Total incoming at a DC = Total outgoing from a DC

In [288]:
incoming = 0
outgoing = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_port_city)):
    incoming = incoming + x_var[f"x[{i+len(df_DC_city)*j}]"]
  for k in range(0, len(df_customer_city)):
    outgoing = outgoing + y_var[f"y[{k+len(df_customer_city)*i}]"]
  model.addConstr(incoming == outgoing)

Pre-work for objective function

In [289]:
p = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_port_city)):
    p = p + df_distance_port_DC['Distance'][i+len(df_DC_city)*j]*x_var[f"x[{i+len(df_DC_city)*j}]"]*z_var[f"z[{i}]"]
p = p*c

In [290]:
q = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_customer_city)):
    q = q + df_distance_DC_customer['Distance'][len(df_customer_city)*i+j]*y_var[f"y[{len(df_customer_city)*i+j}]"]*z_var[f"z[{i}]"]
q = q*c

In [291]:
s = 0
for i in range(0, len(df_DC_city)):
  s = s + df_DC_city['Fixed Cost'][i]*z_var[f"z[{i}]"]

Objective function

In [292]:
objective =  p + q + s

In [293]:
model.setObjective(objective, GRB.MINIMIZE)

In [294]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license 2574149 - for non-commercial use only - registered to pi___@gmail.com
Optimize a model with 192 rows, 1548 columns and 1566 nonzeros
Model fingerprint: 0xed8317d7
Model has 87 quadratic objective terms
Model has 12 quadratic constraints
Variable types: 0 continuous, 1548 integer (18 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+07, 1e+07]
  QObjective range [8e+03, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
  QRHS range       [3e+04, 3e+04]

MIP start from previous solve produced solution with objective 1.51488e+09 (0.05s)
MIP start from previous solve produced solution with objective 1.47221e+09 (0.14s)
Loaded MIP start from previ

In [295]:
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {model.ObjVal}")
else:
    print("No optimal solution found.")

Optimal objective value: 1472214640.0


In [296]:
l = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_port_city)):
    l = l + df_distance_port_DC['Distance'][i+len(df_DC_city)*j]*x_var[f"x[{i+len(df_DC_city)*j}]"].x*z_var[f"z[{i}]"].x
l = l*c
m = 0
for i in range(0, len(df_DC_city)):
  for j in range(0, len(df_customer_city)):
    m = m + df_distance_DC_customer['Distance'][len(df_customer_city)*i+j]*y_var[f"y[{len(df_customer_city)*i+j}]"].x*z_var[f"z[{i}]"].x
m = m*c
t = 0
for i in range(0, len(df_DC_city)):
  t = t + df_DC_city['Fixed Cost'][i]*z_var[f"z[{i}]"].x
print(f"Cost of shipping form port to DCs  {l/100000}")
print(f"Cost of shipping from DCs to customers {m/100000}")
print(f"Cost of opening DC {t/100000}")
print(f"Total cost {(l+m+t)/100000}")

Cost of shipping form port to DCs  4401.7688
Cost of shipping from DCs to customers 10120.377599999998
Cost of opening DC 200.0
Total cost 14722.146399999998


In [297]:
for i in range(0, len(df_DC_city)):
  print(z_var[f"z[{i}]"])

<gurobi.Var z[0] (value 1.0)>
<gurobi.Var z[1] (value 1.0)>
<gurobi.Var z[2] (value 0.0)>


Visualization

In [341]:
!pip install folium
import folium

In [342]:
lat = 0
long = 0
n = 0
for i in range(0, len(df_coordinate_DC_city)):
  lat = lat + df_coordinate_DC_city['Latitude'][i]*z_var[f"z[{i}]"].x
  long = long + df_coordinate_DC_city['Longitude'][i]*z_var[f"z[{i}]"].x
  n = n + z_var[f"z[{i}]"].x
lat = lat/n
long = long/n
m = folium.Map(location=[lat, long], zoom_start=5.1)

In [343]:
for i in range(0, len(df_port_city)):
    folium.Marker(
        location=[df_coordinate_port_city['Latitude'][i], df_coordinate_port_city['Longitude'][i]],
        popup=f"{df_coordinate_port_city['City'][i]}",
        icon=folium.Icon(color="darkgreen", icon="ship", prefix="fa")
    ).add_to(m)

In [344]:
for i in range(0, len(df_customer_city)):
    folium.Marker(
        location=[df_coordinate_customer_city['Latitude'][i], df_coordinate_customer_city['Longitude'][i]],
        popup=f"{df_coordinate_customer_city['City'][i]}",
        icon=folium.Icon(color="red", icon="user", prefix="fa")
    ).add_to(m)

In [345]:
for i in range(0, len(df_DC_city)):
    if z_var[f"z[{i}]"].x == 1:
      folium.Marker(location=[df_coordinate_DC_city['Latitude'][i], df_coordinate_DC_city['Longitude'][i]], popup=f"{df_coordinate_DC_city['City'][i]}",
        icon=folium.Icon(color="black", icon="warehouse", prefix="fa")).add_to(m)

In [346]:
for i in range(0, len(df_port_city)*len(df_DC_city)):
  if z_var[f"z[{i}]"].x == 1:
    sourc_city = df_distance_port_DC['Source'][i]
    destination_city = df_distance_port_DC['Destination'][i]
    source_lat = df_coordinate_port_city.loc[df_coordinate_port_city['City'] == sourc_city, 'Latitude'].iloc[0]
    source_lon = df_coordinate_port_city.loc[df_coordinate_port_city['City'] == sourc_city, 'Longitude'].iloc[0]
    destination_lat = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == destination_city, 'Latitude'].iloc[0]
    destination_lon = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == destination_city, 'Longitude'].iloc[0]
    quantity = x_var[f"x[{i}]"].x
    folium.PolyLine(
        locations=[(source_lat, source_lon), (destination_lat, destination_lon)],
        color="darkgreen",
        popup=folium.Popup(f"Quantity Flow: {quantity}", max_width=400)
    ).add_to(m)

In [348]:
for i in range(0, len(df_DC_city)*len(df_customer_city)):
  if y_var[f"y[{i}]"].x > 1:
    source_city = df_distance_DC_customer['Source'][i]
    destination_city = df_distance_DC_customer['Destination'][i]
    source_lat = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == source_city, 'Latitude'].iloc[0]
    source_lon = df_coordinate_DC_city.loc[df_coordinate_DC_city['City'] == source_city, 'Longitude'].iloc[0]
    destination_lat = df_coordinate_customer_city.loc[df_coordinate_customer_city['City'] == destination_city, 'Latitude'].iloc[0]
    destination_lon = df_coordinate_customer_city.loc[df_coordinate_customer_city['City'] == destination_city, 'Longitude'].iloc[0]

    quantity = y_var[f"y[{i}]"].x
    folium.PolyLine(
            locations=[(source_lat, source_lon), (destination_lat, destination_lon)],
            color="orange",
            popup=folium.Popup(f"Quantity Flow: {quantity}", max_width=200)
        ).add_to(m)

In [350]:
m.save("my_map.html")

In [349]:
m

In [351]:
df_distance_port_DC.to_excel('distance_port_DC.xlsx', index=False)
df_distance_DC_customer.to_excel('distance_DC_customer.xlsx', index=False)